In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
import pandas as pd
import joblib
import pickle
from datetime import datetime

# Example of connection with database

In [15]:
database = ''
user = ''
password = ''
host = ''
port = 0

In [16]:
CONNECTION = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Example of input data

In [17]:
user_id = 1000
time = datetime(2020, 11, 15, 12, 30)
limit = 5

# Reading data

In [18]:
user_features = pd.read_sql(
    f""" SELECT * 
        FROM public.user_data
        WHERE user_id = {user_id}      
    """,
    con=CONNECTION
)

user_features = user_features.drop('user_id', axis=1)
user_features

,gender,age,country,city,exp_group,os,source
0,1,46,Turkey,Kayseri,2,Android,ads


In [19]:
post_features = pd.read_sql(
    """SELECT * FROM public.post_processed_features""",   
    con=CONNECTION
)

content = post_features[['post_id', 'text', 'topic']]
post_features = post_features.drop('text', axis=1)
post_features = post_features.set_index('post_id')
post_features.head()

,topic,TotalTfIdf,MaxTfIdf,MeanTfIdf,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster
post_id,,,,,,,,,,,,,,,,,
1,business,8.189184,0.508965,0.000214,10,0.446862,0.449783,0.511725,0.454039,0.431718,0.480038,0.486540,0.543321,0.561144,0.398347,0.146891,0.487810
2,business,11.169689,0.322813,0.000292,9,0.308796,0.305836,0.390665,0.315877,0.286911,0.352257,0.355370,0.295984,0.460883,0.194327,0.297965,0.357827
3,business,11.939820,0.262184,0.000312,10,0.335695,0.340267,0.409785,0.353175,0.315173,0.392140,0.380555,0.506242,0.477769,0.261311,0.137949,0.403610
4,business,6.138186,0.559407,0.000160,9,0.300260,0.288413,0.385426,0.322719,0.261154,0.349763,0.350393,0.458251,0.408722,0.183320,0.259922,0.366145
5,business,6.019855,0.432699,0.000157,9,0.177063,0.233884,0.301383,0.235845,0.133874,0.249478,0.262709,0.400508,0.384912,0.098322,0.300176,0.278621


In [20]:
"""in order to avoid showing posts the second time to users"""

post_user_liked = pd.read_sql(
    f""" SELECT * 
        FROM public.feed_data as fa 
        WHERE action = 'like' and user_id = {user_id}
        """,
    con=CONNECTION
)

post_user_liked.head()

,timestamp,user_id,post_id,action,target
0,2021-10-17 17:59:39,1000,6499,like,0
1,2021-10-05 10:02:26,1000,6799,like,0
2,2021-10-05 10:06:59,1000,117,like,0
3,2021-10-05 10:14:12,1000,3172,like,0
4,2021-10-05 10:16:28,1000,3842,like,0


# Zipping user and post features

In [21]:
user_features_dict = dict(zip(user_features.columns, user_features.values[0]))
post_user_features = post_features.assign(**user_features_dict)
post_user_features.head()

,topic,TotalTfIdf,MaxTfIdf,MeanTfIdf,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,...,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,gender,age,country,city,exp_group,os,source
post_id,,,,,,,,,,,,,,,,,,,,,
1,business,8.189184,0.508965,0.000214,10,0.446862,0.449783,0.511725,0.454039,0.431718,...,0.398347,0.146891,0.487810,1,46,Turkey,Kayseri,2,Android,ads
2,business,11.169689,0.322813,0.000292,9,0.308796,0.305836,0.390665,0.315877,0.286911,...,0.194327,0.297965,0.357827,1,46,Turkey,Kayseri,2,Android,ads
3,business,11.939820,0.262184,0.000312,10,0.335695,0.340267,0.409785,0.353175,0.315173,...,0.261311,0.137949,0.403610,1,46,Turkey,Kayseri,2,Android,ads
4,business,6.138186,0.559407,0.000160,9,0.300260,0.288413,0.385426,0.322719,0.261154,...,0.183320,0.259922,0.366145,1,46,Turkey,Kayseri,2,Android,ads
5,business,6.019855,0.432699,0.000157,9,0.177063,0.233884,0.301383,0.235845,0.133874,...,0.098322,0.300176,0.278621,1,46,Turkey,Kayseri,2,Android,ads


# Adding time information

In [22]:
post_user_features['hour'] = time.hour
post_user_features['dayofweek'] = time.weekday()
post_user_features['month'] = time.month
post_user_features.head()

,topic,TotalTfIdf,MaxTfIdf,MeanTfIdf,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,...,gender,age,country,city,exp_group,os,source,hour,dayofweek,month
post_id,,,,,,,,,,,,,,,,,,,,,
1,business,8.189184,0.508965,0.000214,10,0.446862,0.449783,0.511725,0.454039,0.431718,...,1,46,Turkey,Kayseri,2,Android,ads,12,6,11
2,business,11.169689,0.322813,0.000292,9,0.308796,0.305836,0.390665,0.315877,0.286911,...,1,46,Turkey,Kayseri,2,Android,ads,12,6,11
3,business,11.939820,0.262184,0.000312,10,0.335695,0.340267,0.409785,0.353175,0.315173,...,1,46,Turkey,Kayseri,2,Android,ads,12,6,11
4,business,6.138186,0.559407,0.000160,9,0.300260,0.288413,0.385426,0.322719,0.261154,...,1,46,Turkey,Kayseri,2,Android,ads,12,6,11
5,business,6.019855,0.432699,0.000157,9,0.177063,0.233884,0.301383,0.235845,0.133874,...,1,46,Turkey,Kayseri,2,Android,ads,12,6,11


# Predicting that post will be liked

In [23]:
"""for example, take control model"""
model = pickle.load(open('model_control_1.pkl', 'rb'))

In [24]:
prediction_liked = model.predict_proba(post_user_features)[:, 1]
post_user_features['predict'] = prediction_liked

# Deleting liked posts

In [25]:
posts_already_liked_ids = list(post_user_liked['post_id'].values)
post_user_features_unliked = post_user_features[~post_user_features.index.isin(posts_already_liked_ids)]
posts_for_recommendation_ids = list(post_user_features_unliked.sort_values('predict')[-limit:].index)

# Response from "server"

In [26]:
posts_for_recommendation_ids

[713, 709, 1766, 1730, 1473]

In [27]:
response_content = content.loc[posts_for_recommendation_ids]
response_content

,post_id,text,topic
713,542,"U2s desire to be number one\n\nU2, who have wo...",entertainment
709,538,Euro firms miss out on optimism\n\nMore than 9...,business
1766,1294,Kilroy-Silk quits shameful UKIP\n\nEx-chat sho...,politics
1730,1263,Burglar defence guidelines issued\n\nHousehold...,politics
1473,1093,Hitler row over Welsh arts cash\n\nAn artist c...,politics


In [28]:
response_list = []
for i in range(limit):
    response_post_dict = {}
    response_post_dict['id'] = response_content.iloc[i]['post_id']
    response_post_dict['text'] = response_content.iloc[i]['text']
    response_post_dict['topic'] = response_content.iloc[i]['topic']
    response_list.append(response_post_dict)
response_list

[{'id': 542,
  'text': 'U2s desire to be number one\n\nU2, who have won three prestigious Grammy Awards for their hit Vertigo, are stubbornly clinging to their status as one of the biggest bands in the world.\n\nThe most popular groups in the history of rock all have several things in common. The music must be inspired and appeal across generations and be distinctive, if not always groundbreaking. But such success is down to more than music. They have to be compelling performers, charismatic and intelligent enough to make good decisions and keep their feet on the ground. They also have to want it. They have to want to be the biggest band ever and not stop wanting it. The Beatles had it, the Rolling Stones still have it, REM hold onto it and Queen were it in a catsuit. And U2 have it in spades, and keep churning it out. Their new album, How To Dismantle An Atomic Bomb, comes 28 years after the schoolfriends got together in Dublin and 17 years after The Joshua Tree cemented their place o